In [1]:
# Import all needed functions and libraries
import sys
sys.path.append('./util_function/')  # Make sure this path is correct relative to your current working directory.
from function_data_preprocessing import *
from function_model import *
from function_evaluate import *
from util_function.function_model import CNN_Model,Bilstm_Model,CNN_Bilstm_sequential_Model,CNN_Bilstm_parallel_Model
from util_function.function_evaluate import evaluate_and_save_results
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Data Loading
file_path = 'NASA_Battery_dataset/B0005.csv'
data = pd.read_csv(file_path)
# Selecting the target and feature columns for model training
target_column = 'capacity'
# This is the feature using before autoencoder
feature_columns_nonautoencoder = ['charge_inCur_drop_T', 'charge_vol_max_T', 'discharge_outCur_rise_T'] 
# choose 3 linear realtionship features with capacity
feature_columns_autoencoder = data.columns.drop(target_column)
# Extracting the feature and target data
target_data = data[target_column].values
feature_data_nonautoencoder = data[feature_columns_nonautoencoder].values
Autoencoder_data = data[feature_columns_autoencoder].values
# Defining the window size for the model
window_size = 5  # Number of time steps used for prediction
smooth_y = np.load('./datasaved/smoothed_B0005_capacity.npy')
smooth_y_origin = smooth_y
# Creating input-output pairs for model training
X, y = create_windows(feature_data_nonautoencoder, target_data, window_size)
X, smooth_y = create_windows(feature_data_nonautoencoder, smooth_y_origin, window_size)
X_autoencoder, smooth_y = create_windows(Autoencoder_data , smooth_y_origin, window_size)


#Preprocessing the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test, smooth_y_train, smooth_y_test = train_test_split(X, smooth_y, test_size=0.2, shuffle=False)
X_autoencoder_train, X_autoencoder_test, smooth_y_train, smooth_y_test = train_test_split(X_autoencoder, smooth_y, test_size=0.2, shuffle=False)
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)

# Apply the same transformation to the test data
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

scaler_autoencoder = StandardScaler()
X_autoencoder_train_scaled = scaler_autoencoder.fit_transform(X_autoencoder_train.reshape(-1, X_autoencoder_train.shape[-1])).reshape(X_autoencoder_train.shape)
X_autoencoder_test_scaled = scaler_autoencoder.transform(X_autoencoder_test.reshape(-1, X_autoencoder_test.shape[-1])).reshape(X_autoencoder_test.shape)

In [3]:
# 5 dim autoencoder
input_dim = X_autoencoder_train_scaled.shape[-1]  # Assuming 20 features per time step
timesteps = window_size  # Assuming 5 time steps per sequence
# Create the model
autoencoder = Autoencoder_Model(input_dim, timesteps,latent_dim=5)
# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = autoencoder.fit(
    X_autoencoder_train_scaled,  X_autoencoder_train_scaled,
    epochs=5000,
    batch_size=32,
    validation_data=(X_autoencoder_test_scaled,X_autoencoder_test_scaled),
    callbacks=[early_stopping],
    verbose=1
)
# Evaluate the model on the test data using the best epoch's model
loss, metric = autoencoder.evaluate(X_autoencoder_test_scaled, X_autoencoder_test_scaled, verbose=0)  # 'metric' should be replaced with the actual metric name you're using

# Find the best epoch and validation loss
best_epoch = early_stopping.stopped_epoch - early_stopping.patience + 1
val_loss = min(history.history['val_loss'])
print(f"Best Epoch: {best_epoch}")
print(f"Validation Loss: {val_loss}")

# Save the model
autoencoder.save('./pretrained_model/Autoencoder_dim5.h5')


Epoch 1/5000
5/5 [==============================] - 3s 159ms/step - loss: 1.0029 - rmse_percentage: inf - val_loss: 1.3616 - val_rmse_percentage: 103.8600
Epoch 2/5000
5/5 [==============================] - 0s 9ms/step - loss: 1.0011 - rmse_percentage: inf - val_loss: 1.3571 - val_rmse_percentage: 104.5599
Epoch 3/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9935 - rmse_percentage: inf - val_loss: 1.3536 - val_rmse_percentage: 105.2010
Epoch 4/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9970 - rmse_percentage: inf - val_loss: 1.3495 - val_rmse_percentage: 106.0700
Epoch 5/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9948 - rmse_percentage: inf - val_loss: 1.3455 - val_rmse_percentage: 106.9424
Epoch 6/5000
5/5 [==============================] - 0s 9ms/step - loss: 0.9913 - rmse_percentage: inf - val_loss: 1.3422 - val_rmse_percentage: 107.5754
Epoch 7/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9887 -

In [4]:
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim5.h5',custom_objects=custom_objects)
autoencoder = loaded_model
Autoencoder_predictions = autoencoder.predict(X_autoencoder_test_scaled)
Autoencoder_predictions_train = autoencoder.predict(X_autoencoder_train_scaled)
np.save('./datasaved/Autoencoder_predictions_dim5.npy', Autoencoder_predictions)
np.save('./datasaved/Autoencoder_predictions_train_dim5.npy', Autoencoder_predictions_train)

5/5 [==============================] - 0s 1ms/step


In [5]:
Autoencoder_predictions = np.load('./datasaved/Autoencoder_predictions_dim5.npy')
absolute_errors = np.abs(X_autoencoder_test_scaled - Autoencoder_predictions)
np.save('./datasaved/absolute_errors_B0005_Autoencoder_dim5.npy', absolute_errors)
print('\nThis is the evaluation results of Autoencoder_dim5\n')
evaluation_results = evaluate_and_save_results(X_autoencoder_test_scaled, Autoencoder_predictions,file_path='./evaluate_result/B0005_Autoencoder_dim5.txt')


This is the evaluation results of Autoencoder_dim5

MAE: 0.5886400201982201
RMSE: 0.7291781000449624
RMSE Percentage: 638.2508871226877


In [6]:
input_dim = X_autoencoder_train_scaled.shape[-1]  # Assuming 20 features per time step
timesteps = window_size  # Assuming 5 time steps per sequence
# Create the model
autoencoder = Autoencoder_Model(input_dim, timesteps,latent_dim=3)
# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = autoencoder.fit(
    X_autoencoder_train_scaled,  X_autoencoder_train_scaled,
    epochs=5000,
    batch_size=32,
    validation_data=(X_autoencoder_test_scaled,X_autoencoder_test_scaled),
    callbacks=[early_stopping],
    verbose=1
)
# Evaluate the model on the test data using the best epoch's model
loss, metric = autoencoder.evaluate(X_autoencoder_test_scaled, X_autoencoder_test_scaled, verbose=0)  # 'metric' should be replaced with the actual metric name you're using

# Find the best epoch and validation loss
best_epoch = early_stopping.stopped_epoch - early_stopping.patience + 1
val_loss = min(history.history['val_loss'])
print(f"Best Epoch: {best_epoch}")
print(f"Validation Loss: {val_loss}")

# Save the model
autoencoder.save('./pretrained_model/Autoencoder_dim3.h5')

Epoch 1/5000
5/5 [==============================] - 3s 148ms/step - loss: 1.0021 - rmse_percentage: inf - val_loss: 1.3578 - val_rmse_percentage: 100.4485
Epoch 2/5000
5/5 [==============================] - 0s 8ms/step - loss: 1.0001 - rmse_percentage: inf - val_loss: 1.3570 - val_rmse_percentage: 100.3332
Epoch 3/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9991 - rmse_percentage: inf - val_loss: 1.3561 - val_rmse_percentage: 100.2235
Epoch 4/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9991 - rmse_percentage: inf - val_loss: 1.3551 - val_rmse_percentage: 100.1340
Epoch 5/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9969 - rmse_percentage: inf - val_loss: 1.3539 - val_rmse_percentage: 100.0386
Epoch 6/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9973 - rmse_percentage: inf - val_loss: 1.3524 - val_rmse_percentage: 99.9353
Epoch 7/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9966 - 

In [7]:
# load the model
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim3.h5',custom_objects=custom_objects)
autoencoder = loaded_model
Autoencoder_predictions = autoencoder.predict(X_autoencoder_test_scaled)
Autoencoder_predictions_train = autoencoder.predict(X_autoencoder_train_scaled)
np.save('./datasaved/Autoencoder_predictions_dim3.npy', Autoencoder_predictions)
np.save('./datasaved/Autoencoder_predictions_train_dim3.npy', Autoencoder_predictions_train)

5/5 [==============================] - 0s 2ms/step


In [8]:
# Evaluate the model and save the results
Autoencoder_predictions = np.load('./datasaved/Autoencoder_predictions_dim3.npy')
absolute_errors = np.abs(X_autoencoder_test_scaled - Autoencoder_predictions)
np.save('./datasaved/absolute_errors_B0005_Autoencoder_dim3.npy', absolute_errors)
print('\nThis is the evaluation results of Autoencoder_dim3\n')
evaluation_results = evaluate_and_save_results(X_autoencoder_test_scaled, Autoencoder_predictions,file_path='./evaluate_result/B0005_Autoencoder_dim3.txt')


This is the evaluation results of Autoencoder_dim3

MAE: 0.6446146173109639
RMSE: 0.7777327207337322
RMSE Percentage: 586.8387561824537


In [9]:
#10 dim autoencoder
input_dim = X_autoencoder_train_scaled.shape[-1]  # Assuming 20 features per time step
timesteps = window_size  # Assuming 5 time steps per sequence
# Create the model
autoencoder = Autoencoder_Model(input_dim, timesteps,latent_dim=10)
# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = autoencoder.fit(
    X_autoencoder_train_scaled,  X_autoencoder_train_scaled,
    epochs=5000,
    batch_size=32,
    validation_data=(X_autoencoder_test_scaled,X_autoencoder_test_scaled),
    callbacks=[early_stopping],
    verbose=1
)
# Evaluate the model on the test data using the best epoch's model
loss, metric = autoencoder.evaluate(X_autoencoder_test_scaled, X_autoencoder_test_scaled, verbose=0)  # 'metric' should be replaced with the actual metric name you're using

# Find the best epoch and validation loss
best_epoch = early_stopping.stopped_epoch - early_stopping.patience + 1
val_loss = min(history.history['val_loss'])
print(f"Best Epoch: {best_epoch}")
print(f"Validation Loss: {val_loss}")

# Save the model
autoencoder.save('./pretrained_model/Autoencoder_dim10.h5')

Epoch 1/5000
5/5 [==============================] - 3s 140ms/step - loss: 0.9915 - rmse_percentage: inf - val_loss: 1.3293 - val_rmse_percentage: 125.9211
Epoch 2/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9854 - rmse_percentage: inf - val_loss: 1.3222 - val_rmse_percentage: 122.7229
Epoch 3/5000
5/5 [==============================] - 0s 7ms/step - loss: 0.9802 - rmse_percentage: inf - val_loss: 1.3147 - val_rmse_percentage: 119.4942
Epoch 4/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9745 - rmse_percentage: inf - val_loss: 1.3067 - val_rmse_percentage: 116.3645
Epoch 5/5000
5/5 [==============================] - 0s 7ms/step - loss: 0.9694 - rmse_percentage: inf - val_loss: 1.2977 - val_rmse_percentage: 113.3781
Epoch 6/5000
5/5 [==============================] - 0s 7ms/step - loss: 0.9609 - rmse_percentage: inf - val_loss: 1.2861 - val_rmse_percentage: 110.6200
Epoch 7/5000
5/5 [==============================] - 0s 8ms/step - loss: 0.9554 -

In [10]:
# load the model
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim10.h5',custom_objects=custom_objects)
autoencoder = loaded_model
Autoencoder_predictions = autoencoder.predict(X_autoencoder_test_scaled)
Autoencoder_predictions_train = autoencoder.predict(X_autoencoder_train_scaled)
np.save('./datasaved/Autoencoder_predictions_dim10.npy', Autoencoder_predictions)
np.save('./datasaved/Autoencoder_predictions_train_dim10.npy', Autoencoder_predictions_train)

5/5 [==============================] - 0s 2ms/step


In [11]:
# Evaluate the model and save the results
Autoencoder_predictions = np.load('./datasaved/Autoencoder_predictions_dim10.npy')
absolute_errors = np.abs(X_autoencoder_test_scaled - Autoencoder_predictions)
np.save('./datasaved/absolute_errors_B0005_Autoencoder_dim10.npy', absolute_errors)
print('\nThis is the evaluation results of Autoencoder_dim10\n')
evaluation_results = evaluate_and_save_results(X_autoencoder_test_scaled, Autoencoder_predictions,file_path='./evaluate_result/B0005_Autoencoder_dim10.txt')


This is the evaluation results of Autoencoder_dim10

MAE: 0.5445968092455191
RMSE: 0.6951980007990026
RMSE Percentage: 377.40980965279715


In [12]:
#15 dim autoencoder
input_dim = X_autoencoder_train_scaled.shape[-1]  # Assuming 20 features per time step
timesteps = window_size  # Assuming 5 time steps per sequence
# Create the model
autoencoder = Autoencoder_Model(input_dim, timesteps,latent_dim=15)
# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = autoencoder.fit(
    X_autoencoder_train_scaled,  X_autoencoder_train_scaled,
    epochs=5000,
    batch_size=32,
    validation_data=(X_autoencoder_test_scaled,X_autoencoder_test_scaled),
    callbacks=[early_stopping],
    verbose=1
)
# Evaluate the model on the test data using the best epoch's model
loss, metric = autoencoder.evaluate(X_autoencoder_test_scaled, X_autoencoder_test_scaled, verbose=0)  # 'metric' should be replaced with the actual metric name you're using

# Find the best epoch and validation loss
best_epoch = early_stopping.stopped_epoch - early_stopping.patience + 1
val_loss = min(history.history['val_loss'])
print(f"Best Epoch: {best_epoch}")
print(f"Validation Loss: {val_loss}")

# Save the model
autoencoder.save('./pretrained_model/Autoencoder_dim15.h5')

Epoch 1/5000
5/5 [==============================] - 5s 229ms/step - loss: 1.0021 - rmse_percentage: inf - val_loss: 1.3664 - val_rmse_percentage: 107.1459
Epoch 2/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9968 - rmse_percentage: inf - val_loss: 1.3574 - val_rmse_percentage: 111.2333
Epoch 3/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9889 - rmse_percentage: inf - val_loss: 1.3499 - val_rmse_percentage: 114.9826
Epoch 4/5000
5/5 [==============================] - 0s 12ms/step - loss: 0.9847 - rmse_percentage: inf - val_loss: 1.3418 - val_rmse_percentage: 118.6053
Epoch 5/5000
5/5 [==============================] - 0s 10ms/step - loss: 0.9783 - rmse_percentage: inf - val_loss: 1.3327 - val_rmse_percentage: 122.4507
Epoch 6/5000
5/5 [==============================] - 0s 10ms/step - loss: 0.9699 - rmse_percentage: inf - val_loss: 1.3228 - val_rmse_percentage: 127.3834
Epoch 7/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.

In [13]:
# load the model
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim15.h5',custom_objects=custom_objects)
autoencoder = loaded_model
Autoencoder_predictions = autoencoder.predict(X_autoencoder_test_scaled)
Autoencoder_predictions_train = autoencoder.predict(X_autoencoder_train_scaled)
np.save('./datasaved/Autoencoder_predictions_dim15.npy', Autoencoder_predictions)
np.save('./datasaved/Autoencoder_predictions_train_dim15.npy', Autoencoder_predictions_train)

5/5 [==============================] - 0s 2ms/step


In [14]:
# Evaluate the model and save the results
Autoencoder_predictions = np.load('./datasaved/Autoencoder_predictions_dim15.npy')
absolute_errors = np.abs(X_autoencoder_test_scaled - Autoencoder_predictions)
np.save('./datasaved/absolute_errors_B0005_Autoencoder_dim15.npy', absolute_errors)
print('\nThis is the evaluation results of Autoencoder_dim15\n')
evaluation_results = evaluate_and_save_results(X_autoencoder_test_scaled, Autoencoder_predictions,file_path='./evaluate_result/B0005_Autoencoder_dim15.txt')


This is the evaluation results of Autoencoder_dim15

MAE: 0.5414487551936005
RMSE: 0.6896905405090876
RMSE Percentage: 186.05845442365106


In [15]:
#17 dim autoencoder
input_dim = X_autoencoder_train_scaled.shape[-1]  # Assuming 20 features per time step
timesteps = window_size  # Assuming 5 time steps per sequence
# Create the model
autoencoder = Autoencoder_Model(input_dim, timesteps,latent_dim=17)
# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=200, verbose=1, mode='min', restore_best_weights=True)

# Train the model
history = autoencoder.fit(
    X_autoencoder_train_scaled,  X_autoencoder_train_scaled,
    epochs=5000,
    batch_size=32,
    validation_data=(X_autoencoder_test_scaled,X_autoencoder_test_scaled),
    callbacks=[early_stopping],
    verbose=1
)
# Evaluate the model on the test data using the best epoch's model
loss, metric = autoencoder.evaluate(X_autoencoder_test_scaled, X_autoencoder_test_scaled, verbose=0)  # 'metric' should be replaced with the actual metric name you're using

# Find the best epoch and validation loss
best_epoch = early_stopping.stopped_epoch - early_stopping.patience + 1
val_loss = min(history.history['val_loss'])
print(f"Best Epoch: {best_epoch}")
print(f"Validation Loss: {val_loss}")

# Save the model
autoencoder.save('./pretrained_model/Autoencoder_dim17.h5')

Epoch 1/5000
5/5 [==============================] - 5s 237ms/step - loss: 0.9900 - rmse_percentage: inf - val_loss: 1.3385 - val_rmse_percentage: 106.5370
Epoch 2/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9806 - rmse_percentage: inf - val_loss: 1.3205 - val_rmse_percentage: 112.4661
Epoch 3/5000
5/5 [==============================] - 0s 10ms/step - loss: 0.9684 - rmse_percentage: inf - val_loss: 1.3036 - val_rmse_percentage: 117.0408
Epoch 4/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9637 - rmse_percentage: inf - val_loss: 1.2850 - val_rmse_percentage: 121.7464
Epoch 5/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9527 - rmse_percentage: inf - val_loss: 1.2633 - val_rmse_percentage: 127.9040
Epoch 6/5000
5/5 [==============================] - 0s 11ms/step - loss: 0.9395 - rmse_percentage: inf - val_loss: 1.2432 - val_rmse_percentage: 134.9947
Epoch 7/5000
5/5 [==============================] - 0s 12ms/step - loss: 0.

In [16]:
# load the model
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim17.h5',custom_objects=custom_objects)
autoencoder = loaded_model
Autoencoder_predictions = autoencoder.predict(X_autoencoder_test_scaled)
Autoencoder_predictions_train = autoencoder.predict(X_autoencoder_train_scaled)
np.save('./datasaved/Autoencoder_predictions_dim17.npy', Autoencoder_predictions)
np.save('./datasaved/Autoencoder_predictions_train_dim17.npy', Autoencoder_predictions_train)

5/5 [==============================] - 0s 2ms/step


In [17]:
# Evaluate the model and save the results
Autoencoder_predictions = np.load('./datasaved/Autoencoder_predictions_dim17.npy')
absolute_errors = np.abs(X_autoencoder_test_scaled - Autoencoder_predictions)
np.save('./datasaved/absolute_errors_B0005_Autoencoder_dim17.npy', absolute_errors)
print('\nThis is the evaluation results of Autoencoder_dim17\n')
evaluation_results = evaluate_and_save_results(X_autoencoder_test_scaled, Autoencoder_predictions,file_path='./evaluate_result/B0005_Autoencoder_dim17.txt')


This is the evaluation results of Autoencoder_dim17

MAE: 0.5364905801975951
RMSE: 0.6870129089925529
RMSE Percentage: 246.08179238832514


In [18]:
# Save the encoder model to evaluate the storge size
# dim 3
custom_objects = {'rmse': rmse, 'rmse_percentage': rmse_percentage}
loaded_model = load_model('./pretrained_model/Autoencoder_dim3.h5',custom_objects=custom_objects)
autoencoder = loaded_model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-2].output)
encoder.save('./pretrained_model/encoder_dim3.h5')

# dim 5
loaded_model = load_model('./pretrained_model/Autoencoder_dim5.h5',custom_objects=custom_objects)
autoencoder = loaded_model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-2].output)
encoder.save('./pretrained_model/encoder_dim5.h5')

# dim 10
loaded_model = load_model('./pretrained_model/Autoencoder_dim10.h5',custom_objects=custom_objects)
autoencoder = loaded_model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-2].output)
encoder.save('./pretrained_model/encoder_dim10.h5')

# dim 15
loaded_model = load_model('./pretrained_model/Autoencoder_dim15.h5',custom_objects=custom_objects)
autoencoder = loaded_model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-2].output)
encoder.save('./pretrained_model/encoder_dim15.h5')

# dim 17
loaded_model = load_model('./pretrained_model/Autoencoder_dim17.h5',custom_objects=custom_objects)
autoencoder = loaded_model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-2].output)
encoder.save('./pretrained_model/encoder_dim17.h5')


